#### Scopus Data Collection ####

In [ ]:
import numpy as np
import csv
import pandas as pd
import json
import requests 
from sklearn.metrics import mean_squared_error
import scipy.optimize as optim
import matplotlib.pyplot as plt

# defining all functions
def find_nth(haystack, needle, n):  #this function is used to find strings in text   
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def countlist(x): 
    z=x
    count_list=[]
    while z>0:
        n=x-(z-1)
        count_list.append(n)
        z=z-1
    return count_list

def update_and_calculate (tech_name):  #function that automatically calculates all the needed values for curve fitting
    tech_name['Cumulative_Sum']=tech_name['Number_of_Publications'].cumsum() 
    tech_name['Year_initialyear']=tech_name['Year']-(tech_name['Year'].min())
    tech_name['Normalized_Records']=tech_name['Cumulative_Sum']/(tech_name['Cumulative_Sum'].max())
    

def my_logistic(t,a,b): #logistic curve function 
    return 1/(1+a*np.exp(-b*t))


def logistic_curve_fitting(tech_name): #function used to logistic curve fit,calculating MSE and returning S value (*100)
    p0=np.array([0.5,0.5])
    bounds=((0,-np.inf),(np.inf,np.inf))
    x=tech_name.Year_initialyear
    y=tech_name.Normalized_Records
    (a,b),cov=optim.curve_fit(my_logistic,x,y,bounds=bounds,p0=p0) 
    MSE=mean_squared_error(y,my_logistic(x,a,b))
    S=(np.sqrt(MSE))*100
    S_Scopus_values.append(S)
    return S_Scopus_values
    


df_0= pd.read_csv('Techs2query_scopus_updatedqueries.csv',encoding= 'unicode_escape') # reading csv containing technology names and their queries as a datafarme

###the following section will send API requests and automatically extract dates+number of publications###

api_url='https://api.elsevier.com/content/search/scopus'
datelist=list(range(1976,2020)) #specify the date range of your search 
techlist=df_0['Technology'].to_list()
querylist=df_0['Query'].to_list() 
year_list=[]
TotalResults_list=[]
q_list=[]
tech_list=[]
for q in querylist:
    for year in datelist:
            qu='TITLE-ABS-KEY('+q+')' #required format to search in title or abstract by Scopus
            params={'query':(qu),'date':(year),'sort':'pubyear','content':'core','apiKey':'f5844ea038f43bc46bf65a10546c1c31'}
            r=requests.get(api_url,params=params)
            word = r.text
            num_lines = word.count('opensearch:totalResults')
            for i in range(num_lines):
                    year_list.append(year)
                    q_list.append(q)
                    index=querylist.index(q)
                    tech_list.append(techlist[index])
                    index_start=find_nth(r.text,'opensearch:totalResults',i+1)
                    index_end = word.find('"', index_start+26,len(word))
                    TotalResults=word[index_start+26:index_end]
                    TotalResults_list.append(word[index_start+26:index_end])
                    

#Technologies, their recorded year and document count values are saved locally as a csv file
array=np.column_stack((tech_list,q_list,year_list,TotalResults_list)) #combined document count and respective year scraped values into a single csv file 
f = open('Scopus_API_Data.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + ","+ "Query"+ "," +"Year" + "," + "Number_of_Publications" + "\n")
    writer.writerows(array)
f.close()

df= pd.read_csv('Scopus_API_Data.csv',encoding= 'unicode_escape') #csv file converted to dataframe for further analysis



### the following code will perform data cleaning, additional calculations required for non linear regression (curve fitting), and finally apply non linear regression and record the Standard Error of Regression (S) value####

Techs = [ frame for Technology, frame in df.groupby('Technology') ]  
Techs_no_results=[]
Tech_name_list=[]
S_Scopus_values=[]
S_zero_values=[]

i=0
for i in range (len(Techs)):
    y=0
    for j in range (len(Techs[i])): #checking if any of the technologies showed no document count results ( all 0's)
        x=len(Techs[i])
        if Techs[i].iloc[j][3]==0:
            y=y+1
    if y==x: # if/else will only perform data cleaning and non linear regression for technologies with results/records. Technologies with zero results are excluded and saved as seperately
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Techs_no_results.append(Tech_name)
        S_zero='no scientific publications found'
        S_zero_values.append(S_zero)
    else:  
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Tech_name_list.append(Tech_name)
        Techs[i]= Techs[i][Techs[i].Number_of_Publications != 0]
        Techs[i].sort_values(by=['Year'])
        Techs[i]= Techs[i][Techs[i].Year< 2012] #this can be set according to the required range of analysis 
        csv_file_name=Tech_name+'.csv'
        update_and_calculate(Techs[i])
        try:
            logistic_curve_fitting(Techs[i])
        except:
            S='null'
            S_Scopus_values.append(S)
            pass
            
        Techs[i].to_csv(csv_file_name) #this outputs a file containing updated csv files (to save locally as a copy/backup) of each technology containing: tech name,query,year,year-initial year, cumulative records, Normalized records
        
        
        
# A copy of technologies and their calculated S values are saved locally as a csv file.This final csv file will be used for TRL estimates.
Tech_name_list.extend(Techs_no_results)
S_Scopus_values.extend(S_zero_values)
array=np.column_stack((Tech_name_list,S_Scopus_values)) 
f = open('Techs_S_Scopus_Values.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + ","+ "S_Scopus" "\n")
    writer.writerows(array)
f.close()




###The following code is used to automatically plot all technologies and the logistic fits and save a .png copy locally###



def my_logistic(t,a,b): #logistic curve function redefined to include k, this will allow plotting y as Cumulative Sum records instead of Normalized records
    k=data.Cumulative_Sum.max()
    return k/(1+a*np.exp(-b*t))

for i in range (len(Techs)):
    try:
        Tech_name=Techs[i].tail(1)['Technology'].values[0]
        data=Techs[i]
        year=data.Year_initialyear
        records=data.Cumulative_Sum
        x=data.Year
        p0=([0.5,0.5])
        bounds=((0,-np.inf),(np.inf,np.inf))
        (a,b),cov=curve_fit(my_logistic,year,records,p0=p0,bounds=bounds)
        plt.plot(x,my_logistic(year,a,b))
        plt.scatter(x,records)
        plt.title(Tech_name)
        plt.xlabel('Year')
        plt.ylabel('Scientific Publications')
        plt.legend(['Logistic Model','Real data'])
        filename=Tech_name+'.png'
        plt.savefig(filename)
        plt.show()

    except:
        print (Tech_name,"error") #some technologies that have no publications, or lack of fit will be skipped over and printed as "error"
        pass


#### USPTO/PatentsView Data Collection ####

In [ ]:
import numpy as np
import csv
import pandas as pd
import json
import requests 
from sklearn.metrics import mean_squared_error
import scipy.optimize as optim
import matplotlib.pyplot as plt

# defining all functions
def find_nth(haystack, needle, n):  #this function is used to find strings in text   
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def countlist(x): 
    z=x
    count_list=[]
    while z>0:
        n=x-(z-1)
        count_list.append(n)
        z=z-1
    return count_list

def update_and_calculate (tech_name):  #function that automatically calculates all the needed values for curve fitting
    tech_name['Cumulative_Sum']=tech_name['Number_of_Publications'].cumsum() 
    tech_name['Year_initialyear']=tech_name['Year']-(tech_name['Year'].min())
    tech_name['Normalized_Records']=tech_name['Cumulative_Sum']/(tech_name['Cumulative_Sum'].max())
    

def my_logistic(t,a,b): #logistic curve function 
    return 1/(1+a*np.exp(-b*t))


def logistic_curve_fitting(tech_name): #function used to logistic curve fit,calculating MSE and returning S value (*100)
    p0=np.array([0.5,0.5])
    bounds=((0,-np.inf),(np.inf,np.inf))
    x=tech_name.Year_initialyear
    y=tech_name.Normalized_Records
    (a,b),cov=optim.curve_fit(my_logistic,x,y,bounds=bounds,p0=p0) 
    MSE=mean_squared_error(y,my_logistic(x,a,b))
    S=(np.sqrt(MSE))*100
    S_USPTO_values.append(S)
    return S_USPTO_values
    


df_0=pd.read_csv('Techs2query_USPTO_updatedqueries.csv',encoding= 'unicode_escape') # reading csv containing technology names and their queries as a datafarme
techlist=df_0['Technology'].to_list()
df_0=df_0.drop(columns=['Technology'])



### USPTO Automated Query Formation: the following code takes keywords in the csv file and automatically formats them to PatentsView required API request format###

#counter to check for OR statements (i.e. X OR Y), AND (i.e. X AND Y) statements, or ANDOR statements (i.e. X AND (Y OR Z))
OR=0  
AND=0
ANDOR=0
for i in range (len(df_0.columns)):           
    if ("OR_" in df_0.columns[i]):
        OR=OR+1
    if("AND_" in df_0.columns[i]):
         AND=AND+1
    if ("ANDOR" in df_0.columns[i]):
        ANDOR=ANDOR+1


check=0
querylist=[]
query_text='{"_and":['
query_text_or=query_text+'{"_or":['
query_text_andor=''
ANDOR_check=0

#loop that will take every term and tranform it to PatentsView's required format to create a final query 
for j in range (len(df_0)):
    #check=0
    for i in range (len(df_0.columns)):   
        
        if ("OR_" in df_0.columns[i] and df_0.iloc[j][i]!='""'):
                

                term=df_0.iloc[j][i]
                query_text_or=query_text_or+'{"_or":[{"_text_phrase":{"patent_abstract":'+term+'}}'+',{"_text_phrase":{"patent_title":'+term+'}}]},'
           
        if ((i==OR-1) and '[{"_or":[{"_or":' in query_text_or):
            query_text_or=query_text_or[:-1]
            query_text_or=query_text_or+"]},"
            
        if ("AND_" in df_0.columns[i] and df_0.iloc[j][i]!='""'):
                #print("o:",j,i)
                if ('[{"_or":[{"_or":' in query_text_or):
                    
                    term=df_0.iloc[j][i]
                    query_text_or=query_text_or+'{"_or":[{"_text_phrase":{"patent_abstract":'+term+'}}'+',{"_text_phrase":{"patent_title":'+term+'}}]},'
                else:
                    check=8
                
                    term=df_0.iloc[j][i]
                    print(term)
                    query_text=query_text+'{"_or":[{"_text_phrase":{"patent_abstract":'+term+'}}'+',{"_text_phrase":{"patent_title":'+term+'}}]},'
               
        
        if ("ANDOR" in df_0.columns[i] and df_0.iloc[j][i]!='""'):
            term=df_0.iloc[j][i]
            query_text_andor=query_text_andor+'{"_or":[{"_text_phrase":{"patent_abstract":'+term+'}}'+',{"_text_phrase":{"patent_title":'+term+'}}]},'
            #print (query_text_andor)
            ANDOR_check=1
            
        if ((i==(OR+AND+ANDOR)-1) and ANDOR_check==1):
            query_text_andor='{"_or":['+query_text_andor
            query_text_andor=query_text_andor[:-1]
            query_text_andor=query_text_andor+"]}," 
            query_text_or=query_text_or+query_text_andor
            query_text_andor=''
                           
    
    print(check)
    if (check==0):        
        querylist.append(query_text_or)
    else:
        querylist.append(query_text)
        
    query_text='{"_and":['
    query_text_or=query_text+'{"_or":['
    check=0
    ANDOR_check=0
    


###the following section will send API requests and automatically extract dates+number of publications###

  
TotalResults_list=[] 
tech_list=[]
q_list=[]
date_list=list(range(1976,2020)) #enter the search range here 
date_list_string=[str(s) for s in date_list]
year_list=[]
for q in querylist:
    for year in date_list_string:
        r=requests.get('https://www.patentsview.org/api/patents/query?q='+q+'{"_gte":{"app_date":"'+year+'-01-01"}},{"_lte":{"app_date":"'+year+'-12-31"}}]}&f=["app_date"]')
        word = r.text
        num_lines = word.count('total_patent_count')  
        for i in range(num_lines):
            q_list.append(q)
            index=querylist.index(q)
            tech_list.append(techlist[index])
            year_list.append(year)
            index_start=find_nth(r.text,'total_patent_count',i+1)
            index_end= word.find('"', index_start+20,len(word))
            TotalResults_list.append(word[index_start+20:index_end])
        


                    

#Technologies, their recorded year and document count values are saved locally as a csv file
array=np.column_stack((tech_list,q_list,year_list,TotalResults_list)) #combined document count and respective year scraped values into a single csv file 
f = open('USPTO_API_Data.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + ","+ "Query"+ "," +"Year" + "," + "Number_of_Publications" + "\n")
    writer.writerows(array)
f.close()

df= pd.read_csv('USPTO_API_Data.csv',encoding= 'unicode_escape') #csv file converted to dataframe for further analysis



### the following code will perform data cleaning, additional calculations required for non linear regression (curve fitting), and finally apply non linear regression and record the Standard Error of Regression (S) value####

Techs = [ frame for Technology, frame in df.groupby('Technology') ]  
Techs_no_results=[]
Tech_name_list=[]
S_USPTO_values=[]
S_zero_values=[]
i=0
for i in range (len(Techs)):
    y=0
    for j in range (len(Techs[i])): #checking if any of the technologies showed no document count results ( all 0's)
        x=len(Techs[i])
        if Techs[i].iloc[j][3]==0:
            y=y+1
    if y==x: # if/else will only perform data cleaning and non linear regression for technologies with results/records. Technologies with zero results are excluded and saved as seperately
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Techs_no_results.append(Tech_name)
        S_zero='no patents found'
        S_zero_values.append(S_zero)
    else:  
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Tech_name_list.append(Tech_name)
        Techs[i]= Techs[i][Techs[i].Number_of_Publications != 0]
        Techs[i].sort_values(by=['Year'])
        Techs[i]= Techs[i][Techs[i].Year< 2012] #this can be set according to the required range of analysis 
        csv_file_name=Tech_name+'.csv'
        update_and_calculate(Techs[i])
        try:
            logistic_curve_fitting(Techs[i])
        except:
            S='null'
            S_USPTO_values.append(S)
            pass
            
        Techs[i].to_csv(csv_file_name) #this outputs a file containing updated csv files (to save locally as a copy/backup) of each technology containing: tech name,query,year,year-initial year, cumulative records, Normalized records
        
        
        
# A copy of technologies and their calculated S values are saved locally as a csv file.This final csv file will be used for TRL estimates.
Tech_name_list.extend(Techs_no_results)
S_USPTO_values.extend(S_zero_values)
array=np.column_stack((Tech_name_list,S_USPTO_values)) 
f = open('Techs_S_USPTO_Values.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + ","+ "S_USPTO" "\n")
    writer.writerows(array)
f.close()



###The following code is used to automatically plot all technologies and the logistic fits and save a .png copy locally###



def my_logistic(t,a,b): #logistic curve function redefined to include k, this will allow plotting y as Cumulative Sum records instead of Normalized records
    k=data.Cumulative_Sum.max()
    return k/(1+a*np.exp(-b*t))

for i in range (len(Techs)):
    try:
        Tech_name=Techs[i].tail(1)['Technology'].values[0]
        data=Techs[i]
        year=data.Year_initialyear
        records=data.Cumulative_Sum
        x=data.Year
        p0=([0.5,0.5])
        bounds=((0,-np.inf),(np.inf,np.inf))
        (a,b),cov=curve_fit(my_logistic,year,records,p0=p0,bounds=bounds)
        plt.plot(x,my_logistic(year,a,b))
        plt.scatter(x,records)
        plt.title(Tech_name)
        plt.xlabel('Year')
        plt.ylabel('Patents')
        plt.legend(['Logistic Model','Real data'])
        filename=Tech_name+'.png'
        plt.savefig(filename)
        plt.show()

    except:
        print (Tech_name,"error") #some technologies that have no publications, or lack of fit will be skipped over and printed as "error"
        pass
    
    
    
    
    



### Factiva Data Collection ###

In [ ]:
import numpy as np
import csv
import pandas as pd
import json
import requests 
from sklearn.metrics import mean_squared_error
import scipy.optimize as optim
import matplotlib.pyplot as plt

# defining all functions
def find_nth(haystack, needle, n):  #this function is used to find strings in text   
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def countlist(x): 
    z=x
    count_list=[]
    while z>0:
        n=x-(z-1)
        count_list.append(n)
        z=z-1
    return count_list


def find_in_list_of_list(mylist, char): 
    for sub_list in mylist:
        if char in sub_list:
            x=mylist.index(sub_list)
            y=sub_list.index(char)
            return (x,y)
            
    raise ValueError("'{char}' is not in list".format(char = char))
    

def returnNotMatches(a, b): #function will be used to flag queries that don't match ( will return non matches)
    return [[x for x in a if x not in b], [x for x in b if x not in a]]

def update_and_calculate (tech_name):  #function that automatically calculates all the needed values for curve fitting
    tech_name['Cumulative_Sum']=tech_name['Number_of_Publications'].cumsum() 
    tech_name['Year_initialyear']=tech_name['Year']-(tech_name['Year'].min())
    tech_name['Normalized_Records']=tech_name['Cumulative_Sum']/(tech_name['Cumulative_Sum'].max())
    

def my_logistic(t,a,b): #logistic curve function 
    return 1/(1+a*np.exp(-b*t))


def logistic_curve_fitting(tech_name): #function used to logistic curve fit,calculating MSE and returning S value (*100)
    p0=np.array([0.5,0.5])
    bounds=((0,-np.inf),(np.inf,np.inf))
    x=tech_name.Year_initialyear
    y=tech_name.Normalized_Records
    (a,b),cov=optim.curve_fit(my_logistic,x,y,bounds=bounds,p0=p0) 
    MSE=mean_squared_error(y,my_logistic(x,a,b))
    S=(np.sqrt(MSE))*100
    S_Factiva_values.append(S)
    return S_Factiva_values
    
df_0= pd.read_csv('Techs2query_Factiva_updatedqueries.csv',encoding= 'unicode_escape')  # reading csv containing technology names and their queries as a datafarme
techlist=df_0['Technology'].to_list()
querylist=df_0['Query'].to_list()





### The following code will automatically read every csv file of each technology, tranform into a dictionary, and extract document counts, year, and query.

filenames = glob(r'C:\Users\Safa\A_Research_UofT\Data Collection\Data Collection_Factiva\Tech_CSV_files\ChartSummary*.csv')
Year_list_cleaned=[]
Tech_query_list=[]
Tech_query_short=[]
Documentcount_list=[]
Technology_list=[]
for f in filenames:
    with open(f, 'r') as read_obj:
        dictio=[] 
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
        for row in csv_reader: # row variable is a list that represents a row in csv
            dictio.append(row)

    
    Year_list=[]
    (a,b)=find_in_list_of_list(dictio, 'Date') #locating index of each term 
    (c,d)=find_in_list_of_list(dictio, 'Search Summary')
    (e,f)=find_in_list_of_list(dictio,'Text ')
    Tech_query=dictio[e][f+1]

    
    i=5
   
        
    for row in dictio:
        while (a-2)<i<(c-2):
            Year_list.append(dictio[i][0])
            Documentcount_list.append(dictio[i][1])
            i=i+1
    i=0
    for row in Year_list:
        index_start_t=find_nth(row,'Start Date',i+1)
        index_end_t = row.find(' End Date', index_start_t+22,len(row))
        TimeBin=row[index_start_t+22:index_end_t]
        Year_list_cleaned.append(row[index_start_t+22:index_end_t])
        i=i+1
        Tech_query_list.append(Tech_query)
        
        index=df_0[df_0['Query']==Tech_query].index.values #locating the index of the technology name by its query 
        index.astype(int)
        x=index[0]
        Techname=df_0.iloc[x][0]
        Technology_list.append(Techname)

    
    
#part of code that flags mistakes in queries
returnNotMatches(Tech_query_list, querylist)


#Technologies, their recorded year and document count values are saved locally as a csv file
array=np.column_stack((Technology_list,Tech_query_list,Year_list_cleaned,Documentcount_list)) #combined document count and respective year scraped values into a single csv file and outputs it so that a physical copy is available prior to further analysis
f = open('Factiva_API_Data.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + "," +"Query"+ ","+ "Year" + "," + "Number_of_Publications" + "\n")
    writer.writerows(array)
f.close()
df= pd.read_csv('Factiva_API_Data.csv',encoding= 'unicode_escape')  #csv file converted to dataframe for further analysis




### the following code will perform data cleaning, additional calculations required for non linear regression (curve fitting), and finally apply non linear regression and record the Standard Error of Regression (S) value####

Techs = [ frame for Technology, frame in df.groupby('Technology') ]  
Techs_no_results=[]
Tech_name_list=[]
S_Factiva_values=[]
S_zero_values=[]
i=0
for i in range (len(Techs)):
    y=0
    for j in range (len(Techs[i])): #checking if any of the technologies showed no document count results ( all 0's)
        x=len(Techs[i])
        if Techs[i].iloc[j][3]==0:
            y=y+1
    if y==x: # if/else will only perform data cleaning and non linear regression for technologies with results/records. Technologies with zero results are excluded and saved as seperately
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Techs_no_results.append(Tech_name)
        S_zero='no patents found'
        S_zero_values.append(S_zero)
    else:  
        Tech_name=Techs[i].tail(1)['Technology'].values[0] 
        Tech_name_list.append(Tech_name)
        Techs[i]= Techs[i][Techs[i].Number_of_Publications != 0]
        Techs[i].sort_values(by=['Year'])
        Techs[i]= Techs[i][Techs[i].Year< 2012] #this can be set according to the required range of analysis 
        csv_file_name=Tech_name+'.csv'
        update_and_calculate(Techs[i])
        try:
            logistic_curve_fitting(Techs[i])
        except:
            S='null'
            S_Factiva_values.append(S)
            pass
            
        Techs[i].to_csv(csv_file_name) #this outputs a file containing updated csv files (to save locally as a copy/backup) of each technology containing: tech name,query,year,year-initial year, cumulative records, Normalized records
        
        
        
# A copy of technologies and their calculated S values are saved locally as a csv file.This final csv file will be used for TRL estimates.
Tech_name_list.extend(Techs_no_results)
S_Factiva_values.extend(S_zero_values)
array=np.column_stack((Tech_name_list,S_Factiva_values)) 
f = open('Techs_S_Factiva_Values.csv', 'w')
with f:
    writer=csv.writer(f)
    f.write("Technology" + ","+ "S_Factiva" "\n")
    writer.writerows(array)
f.close()



###The following code is used to automatically plot all technologies and the logistic fits and save a .png copy locally###



def my_logistic(t,a,b): #logistic curve function redefined to include k, this will allow plotting y as Cumulative Sum records instead of Normalized records
    k=data.Cumulative_Sum.max()
    return k/(1+a*np.exp(-b*t))

for i in range (len(Techs)):
    try:
        Tech_name=Techs[i].tail(1)['Technology'].values[0]
        data=Techs[i]
        year=data.Year_initialyear
        records=data.Cumulative_Sum
        x=data.Year
        p0=([0.5,0.5])
        bounds=((0,-np.inf),(np.inf,np.inf))
        (a,b),cov=curve_fit(my_logistic,year,records,p0=p0,bounds=bounds)
        plt.plot(x,my_logistic(year,a,b))
        plt.scatter(x,records)
        plt.title(Tech_name)
        plt.xlabel('Year')
        plt.ylabel('News Records')
        plt.legend(['Logistic Model','Real data'])
        filename=Tech_name+'.png'
        plt.savefig(filename)
        plt.show()

    except:
        print (Tech_name,"error") #some technologies that have no publications, or lack of fit will be skipped over and printed as "error"
        pass
    
    
    
    
    

